In [8]:
def test_infer(model_name,
               input_data,
               headers=None,
               request_compression_algorithm=None,
               response_compression_algorithm=None):
    inputs = []
    outputs = []
    inputs.append(httpclient.InferInput('input_2', [1, 1, 1, 4], "FP32"))

    # Initialize the data
    inputs[0].set_data_from_numpy(input_data)

    outputs.append(httpclient.InferRequestedOutput('dense_10/Softmax'))
    query_params = {'test_1': 1}
    
    triton_client = httpclient.InferenceServerClient(
                url="localhost:8000",
                verbose=1)
    
    results = triton_client.infer(
        model_name,
        inputs,
        outputs=outputs,
        query_params=query_params,
        headers=headers,
        request_compression_algorithm=request_compression_algorithm,
        response_compression_algorithm=response_compression_algorithm)
    # print(outputs)

    # print(results)
    return results

In [2]:
#Import required libraries 
import keras #library for neural network
import pandas as pd #loading data in table form  
import seaborn as sns #visualisation 
import matplotlib.pyplot as plt #visualisation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import normalize #machine learning algorithm library

#  Load the dataset, which contains the data points(sepal length, petal length, etc) and corresponding labels(type of iris)
iris_dataset=pd.read_csv("https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv")

iris_dataset.loc[iris_dataset["species"]=="setosa","species"]=0
iris_dataset.loc[iris_dataset["species"]=="versicolor","species"]=1
iris_dataset.loc[iris_dataset["species"]=="virginica","species"]=2

# #This is a debug statement to make sure we uploaded the dataset correctly. 
# #We can comment it out when we actually run the code.
# #print(iris_dataset)

2022-09-12 20:16:41.867880: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-12 20:16:42.015777: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-12 20:16:42.021026: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-12 20:16:42.021042: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from keras.utils import np_utils


# Break the dataset up into the examples (X) and their labels (y)
X = iris_dataset.iloc[:, 0:4].values
y = iris_dataset.iloc[:, 4].values
X=normalize(X,axis=0)

# Split up the X and y datasets randomly into train and test sets
# 20% of the dataset will be used for the test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=31)

#Change the label to one hot vector
'''
[0]--->[1 0 0]
[1]--->[0 1 0]
[2]--->[0 0 1]
'''
y_train=np_utils.to_categorical(y_train,num_classes=3)
y_test=np_utils.to_categorical(y_test,num_classes=3)

In [ ]:
import h5py
file = h5py.File('A_deploy_model/saved_model/h5/model.h5')

layer_names = file.keys()

In [ ]:
from keras.models import load_model

classifier=load_model('A_deploy_model/saved_model/h5/model.h5')

classifier.summary()

In [11]:
data = X_test[5].astype(np.float32)
input_ = np.reshape(data, (1, 1, 1, 4))
result = test_infer(model_name="bloom_classification", input_data = input_)
print(result.as_numpy('dense_10/Softmax'))

POST /v2/models/bloom_classification/infer?test_1=1, headers {'Inference-Header-Content-Length': 178}
b'{"inputs":[{"name":"input_2","shape":[1,1,1,4],"datatype":"FP32","parameters":{"binary_data_size":16}}],"outputs":[{"name":"dense_10/Softmax","parameters":{"binary_data":true}}]}\xf6\x02\x88=?\xa4\xa2=\x1e\xa9\xe1<KY\x8d<'
<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/octet-stream', 'inference-header-content-length': '170', 'content-length': '182'}>
bytearray(b'{"model_name":"bloom_classification","model_version":"1","outputs":[{"name":"dense_10/Softmax","datatype":"FP32","shape":[1,3,1,1],"parameters":{"binary_data_size":12}}]}')
[[[[0.9445197 ]]

  [[0.05217379]]

  [[0.0033065 ]]]]


In [5]:
import argparse
import numpy as np
import sys
import gevent.ssl

import tritonclient.http as httpclient
from tritonclient.utils import InferenceServerException

In [ ]:
!ls